In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys

sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt

from src.core import Domain1D, Domain2D
from src.solvers import Wave, Heat
from src.components import Listener, RickerSource, HarmonicSource
from src.visualization import PhysicsAnimator
import src.utils as utils

In [ ]:
room = Domain2D(length =[10.0,10.0], dx=0.05)

speaker_center = [2.0, 5.0]
speaker_wall_width = 0.1
speaker_length = 1.0

def add_speaker_cabinet(room, center, length, wall_width):
    room.add_rectangular_obstacle(
        pos = center,
        size = [wall_width, length]
    )

    room.add_rectangular_obstacle(
        pos = [center[0] + length/2, center[1] + length/2],
        size = [length, wall_width]
    )

    room.add_rectangular_obstacle(
        pos = [center[0] + length/2, center[1] - length/2 + wall_width/2],
        size = [length, wall_width]
    )


room = Domain2D(length =[10.0,10.0], dx=0.05)

speaker_center = [2.0, 5.0]
speaker_wall_width = 0.1
speaker_length = .5

add_speaker_cabinet(room, speaker_center, speaker_length, speaker_wall_width)

# room.add_circular_cavity(
#     pos = [speaker_center[0]+3, speaker_center[1]],
#     radius = 0.5
# )

plt.imshow(room.mask)

In [ ]:
# 3. Setup Solver
# Note: Using default Neumann (Hard) boundaries for the obstacles
solver = Wave(domain=room, c=343.0, boundary_type='robin', R = 0.0)

# 4. Place Source DEEP inside the box
# src = HarmonicSource(pos=[speaker_center[0] + room.ds[0], speaker_center[1]], frequency=200, amplitude=100.0)
# solver.add_dynamic_source(src)

src = RickerSource(pos=[speaker_center[0] + room.ds[0], speaker_center[1]], peak_freq=2000, delay=0.005, amplitude=100.0)
solver.add_dynamic_source(src)

# Listener A: "On-Axis" (Directly in the beam)
mic_on = Listener(pos=[speaker_center[0] + 3, speaker_center[1]])
solver.add_listener(mic_on)

# Listener B: "Off-Axis" (In the shadow zone)
mic_off = Listener(pos=[speaker_center[0] - 1, speaker_center[1]-3]
)
solver.add_listener(mic_off)

# 5. Run and Animate
animator = PhysicsAnimator(solver, total_time=0.15)
animator.run()

# Create animation (Visualizing the 'beam' or 'wrap')
fig = animator.create_animation(skip_frames=5)
fig.show()

In [ ]:
ts1, sig1 = mic_on.get_time_series()
ts2, sig2 = mic_off.get_time_series()

plt.plot(ts1, sig1 ,alpha = 0.8, label = 'mic_on')
plt.plot(ts2, sig2, alpha = 0.8, label = 'mic_off')
plt.legend()